In [1]:
import cvxpy as cp
import numpy as np
import random

In [2]:
np.random.beta(1, 5, 1).item()*50 + 20

34.53105514529036

In [3]:
def queueGenerator(numQueue):
    listDataEquip, listDataPcard, listDataPtime, listDataPriority = [], [], [], []
    listEquip = ['4082', 'V93K']
    listPcard = ['AV44', 'FF44']

    for _ in range(numQueue):
        equip = random.choices(listEquip, weights=[0.8, 0.2])
        pcard = random.choices(listPcard, weights=[0.8, 0.2])
        # 0~100까지
        ptime = np.random.beta(1, 5, 1).item()*100
        priority = random.choices(range(0, 7), weights=[0.8, 0.01, 0.01, 0.01, 0.02, 0.05, 0.1])
        listDataEquip.append(equip)
        listDataPcard.append(pcard)
        listDataPtime.append(ptime)
        listDataPriority.append(priority)
    return listDataEquip, listDataPcard, listDataPtime, listDataPriority

numQueue = int(np.random.beta(1, 5, 1).item()*50 + 30)
listDataEquip, listDataPcard, p, priority = queueGenerator(numQueue)

In [ ]:
# Constants

J = len(listDataEquip)
M = 14
setup_time = 0.13

flagEquip = np.array([[listDataEquip[j] == listDataEquip[m] for j in range(J)] for m in range(J)])
flagPcard = np.array([[listDataPcard[j] == listDataPcard[m] for j in range(J)] for m in range(J)])
# Variables
C = cp.Variable(J)
z = cp.Variable((J, M), boolean=True)
o = cp.Variable((J, J, M), boolean=True)
# delta = cp.Variable((J, J, M), boolean=True)  # Indicates part mismatch when j follows i on m

constraints = []
big_M = 1000

# Job assignment
for j in range(J):
    constraints.append(cp.sum(z[j, :]) == 1)
    constraints.append(C[j] >= p[j])

# Compatibility
for j in range(J):
    for m in range(M):
        if j < M:
            if j == m:
                constraints.append(z[j, m] == 1)
            for k in range(M, J):
                constraints.append(C[j] <= C[k] + big_M * (1 - z[k, m]))
        elif not flagEquip[j, m]:
            constraints.append(z[j, m] == 0)

# Disjunctive constraints + setup time
for j in range(J):
    for k in range(J):
        if j != k:
            for m in range(M):
                same_machine = z[j, m] + z[k, m] - 1  # 1 if both assigned to m
                parts_different = int(listDataPcard[j] != listDataPcard[k])
                constraints.append(C[j] <= C[k] - p[k] - setup_time * parts_different
                                   + big_M * (1 - o[j, k, m] + 1 - same_machine))

                constraints.append(C[k] <= C[j] - p[j] - setup_time * parts_different
                                   + big_M * (o[j, k, m] + 1 - same_machine))

                constraints.append(o[j, k, m] + o[k, j, m] <= 1)

# Objective: minimize total completion time
objective = cp.Minimize(cp.max(C))

# Solve
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.HIGHS)

# Print results
print('status: ', problem.status)
print('value: ', problem.value)
print("총 완료 시간:", np.max(C.value))
for j in range(J):
    print(f"Job {j}: Start = {C.value[j]:.2f - p[j]:.2f - (setup_time * int(listDataPcard[j] != listDataPcard[j-1]) if j != 0 else 0) }, End = {C.value[j]:.2f}, Machine = {np.argmax(z.value[j])}")

c:\Users\ragna\AppData\Local\Programs\Python\Python311\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:418: UserWarning: The problem has an expression with dimension greater than 2. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(


In [ ]:
for j in range(J):
    print(f'{C.value[j]:.2f}')

15.63
32.35
25.74
10.63
35.23
23.41
29.23
9.10
4.86
13.54
1.42
3.29
16.62
0.48
39.00
35.23
49.07
53.52
43.62
48.63
58.43
80.01
61.89
54.91
43.52
49.80
59.86
58.68
39.43
81.97
81.21
81.97
35.23
59.75
41.43
35.23
35.99
35.23
35.23
74.69
35.23
72.97


In [ ]:
import plotly.express as px
import pandas as pd

# 결과 예시
data = [
    {"Job": "Job 0", "Start": 0.0, "Finish": 3.0, "Machine": "Machine 0"},
    {"Job": "Job 1", "Start": 1.0, "Finish": 3.0, "Machine": "Machine 1"},
    {"Job": "Job 2", "Start": 3.0, "Finish": 5.0, "Machine": "Machine 0"},
]

df = pd.DataFrame(data)

# Gantt Chart 생성
fig = px.timeline(df, x_start="Start", x_end="Finish", y="Machine", color="Job", title="Scheduling Gantt Chart")
fig.update_yaxes(autorange="reversed")  # 위쪽에 Machine 0, 아래 Machine 1
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed